In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Comando para descompactar o arquivo ZIP
!unzip "/content/drive/MyDrive/wikipedia-pt-br.zip" -d "/content"

# Atualize o caminho do arquivo JSON descompactado
PATH = "/content/ptwiki-latest.json"

Archive:  /content/drive/MyDrive/wikipedia-pt-br.zip
  inflating: /content/ptwiki-latest.json  y



In [3]:
PATH = "/content/ptwiki-latest.json"
import json

"""
Rafael Mascarenhas Brown de Andrade
Ciência de Dados e Machine Learning
Matutino - Asa Norte - Turma: A
22304454
"""
# Use a codificação detectada para abrir o arquivo
with open(PATH) as f:
    sample = json.loads(f.readline())

for key, value in sample.items():
    print(key, type(value))
    """if key == 'interlinks':
        print(value)"""


title <class 'str'>
section_texts <class 'list'>
section_titles <class 'list'>
interlinks <class 'dict'>


In [4]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=688a7b108f2f95cfdaaf90e87567ef7fbb401bf565120e56ff027a340649464f
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [5]:


from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "25g") \
    .config("spark.driver.memory", "25g") \
    .appName("test") \
    .getOrCreate()

In [6]:
spark

In [7]:
"""from pyspark.sql.types import StructType, StructField, StringType, ArrayType

schema = StructType([
  StructField('title', StringType()),
  StructField('section_texts', ArrayType(StringType())),
  StructField('section_titles', ArrayType(StringType())),
  StructField('interlinks', MapType(StringType(),StringType()))
])
df = spark.read.schema(schema).json(PATH)"""

"from pyspark.sql.types import StructType, StructField, StringType, ArrayType\n\nschema = StructType([\n  StructField('title', StringType()),\n  StructField('section_texts', ArrayType(StringType())), \n  StructField('section_titles', ArrayType(StringType())), \n  StructField('interlinks', MapType(StringType(),StringType())) \n])\ndf = spark.read.schema(schema).json(PATH)"

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType # Import MapType

schema = StructType([
  StructField('title', StringType()),
  StructField('section_texts', ArrayType(StringType())),
  StructField('section_titles', ArrayType(StringType())),
  StructField('interlinks', MapType(StringType(),StringType()))
])
df = spark.read.schema(schema).json(PATH)

In [9]:
df.show(2)

+--------------+--------------------+--------------------+--------------------+
|         title|       section_texts|      section_titles|          interlinks|
+--------------+--------------------+--------------------+--------------------+
|    Astronomia|[\nFormação estre...|[Introduction, Hi...|{Calendário julia...|
|América Latina|[\nA '''América L...|[Introduction, Et...|{Altitude -> Elev...|
+--------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [10]:
def read_texts(data):
  return json.loads(data.value)["section_texts"]

In [11]:


rdd= spark.read.text(PATH).rdd.map(read_texts)

In [12]:
df_teste= df.select("section_texts").limit(3)


In [14]:
import re

START = '<|start|>'
REPLACEMENTS = [
    (r'((?!\w|{START}).)+', ' '),
    (r'\d+', '<|number|>'),
    (r' +', ' '),
]

def clean(text):
    for pattern, repl in REPLACEMENTS:
        text = re.sub(pattern, repl, text)
    return text.lower()

def tokenize(texts):
    if isinstance(texts, str):
        texts = [texts]
    return [token for text in texts for token in [START] + clean(text).split()]


In [15]:
# Correct the import statement from 'collecctions' to 'collections'
from collections import defaultdict


def count_words(texte):
  words = tokenize(texte)
  counts = defaultdict(int)
  for word in words:
    counts[word] += 1
  return list (counts.items())

In [16]:
counts2 = rdd.flatMap(count_words)\
  .reduceByKey(lambda acc,x: acc + x)



In [17]:
counts2 = counts2.cache()
counts2= counts2.sortBy(lambda x: -x[1])


In [18]:
a = counts2.take(30000)

In [19]:
VOCAB_SIZE = 10_000

token2id = {token: i for i, (token, _) in enumerate(a[:VOCAB_SIZE])}
id2token = {i: token for i, (token, _) in enumerate(a[:VOCAB_SIZE])}


In [20]:
 def get_ngrams(text,n=2):
  tokens = tokenize(text)
  return list(zip(*[tokens[i:] for i in range(n)]))

In [21]:
def count_bigrams_in_list(texts, filter_vocab=True):
  ngrams = get_ngrams(texts,n=2)
  counts = defaultdict(lambda: defaultdict(int))
  for ngram in ngrams:
    if filter_vocab or all(token in token2id for token in ngram):
      prev_token, token = ngram
      counts [prev_token][token] += 1
  return counts.items()

In [22]:
def merge_ngrams_count(acc,occ):
  for token in occ.keys():
    acc[token] = acc[token] + occ[token]
  return acc


In [23]:
rdd = rdd.cache()
all_bigrams = rdd.flatMap(count_bigrams_in_list).reduceByKey(merge_ngrams_count).collect()


In [24]:

bigrams_dict = {key: sorted(value.items(), key=lambda x: x[1], reverse=True) for key, value in all_bigrams}




In [25]:
import numpy as np

def generate(text, max_new_tokens=20, top_k=50):

    # Tokenize the input text and get the last token
    last_token = tokenize(text)[-1]

    for i in range(max_new_tokens):
        # Get top K probable next tokens and their weights from the bigrams_dict
        tokens, weights = zip(*bigrams_dict[last_token][:top_k])

        # Normalize the weights to sum up to 1
        weights = np.array(weights) / sum(weights)

        # Randomly select the next token based on the weights
        index = np.argmax(np.random.multinomial(1, weights)).item()

        # Check if the selected token is a valid token (not starting with '<|')
        if not tokens[index].startswith('<|'):
            # Update the last_token and append it to the text
            last_token = tokens[index]
            text += ' ' + last_token

    return text


In [26]:
generate("brasil")

'brasil categoria filmes baseados nessas missões secretas de um dos últimos dias depois de sua última prova de uma'